# Question and Answering

In [1]:
import logging
import json
import pandas as pd
from simpletransformers.question_answering import QuestionAnsweringModel, QuestionAnsweringArgs

In [2]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [3]:
data_dir = "/dataset/indonlu/facqa_qa-factoid-itb"
file_train = f'{data_dir}/train_preprocess.csv'
file_valid = f'{data_dir}/valid_preprocess.csv'
file_test = f'{data_dir}/test_preprocess_masked_label.csv'
#file_labels_map = f'{data_dir}/labels-map.csv'

In [23]:
def save_result(filename, data, texts):
    prediction = {'index':[], 'label':[]}
    for i, qa in enumerate(data):
        id = qa['qas'][0]['id']
        question = qa['qas'][0]['question']    
        context = qa['context']
        label = ['O']*len(context.split())
        try:
            answer = texts['similar_text'][id]['predicted']
            answer_list = answer.split()
            answer_start = context.index(answer)
            context_list = context[:answer_start].split()
            answer_index = len(context_list)
            for j, ans in enumerate(answer_list):
                if j == 0:
                    label[answer_index+j] = 'B'
                else:
                    label[answer_index+j] = 'I'
        except KeyError:
            pass
        prediction['index'].append(i)
        prediction['label'].append(str(label))
    prediction_df = pd.DataFrame.from_dict(prediction).set_index('index')
    prediction_df.to_csv(filename)

In [4]:
data_train = pd.read_csv(file_train, names=['question','passage','seq_label'], header=0)

In [5]:
data_train.head(10)

,question,passage,seq_label
0,"['Kelompok', 'apakah', 'yang', 'menyatakan', '...","['Lewat', 'telepon', 'ke', 'kantor', 'berita',...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
1,"['Di', 'pulau', 'manakah', ',', 'PT', 'Mutiara...","['Sejak', 'tahun', '1980', '-', 'an', 'di', 'K...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
2,"['Apakah', 'warna', 'pasir', 'pada', 'pesisir'...","['Titik', 'kedua', 'adalah', 'Pantai', 'Baturu...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
3,"['Apa', 'nama', 'perusahaan', 'China', 'yang',...","['Sichuan', ',', 'Kompas', '-', 'Pemerintah', ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
4,"['Siapa', 'yang', 'akan', 'memindahkan', 'Patu...","['Jakarta', ',', 'Kompas', '-', 'Patung', 'Kar...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
5,"['Berapa', 'kapasitas', 'PLTS', 'yang', 'sudah...","['Untuk', 'PLTS', ',', 'satu', 'unit', 'PLTS',...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
6,"['Tanggal', 'berapa', 'Presiden', 'Susilo', 'B...","['Jakarta', ',', 'Kompas', '-', 'Presiden', 'S...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
7,"['Siapa', 'nama', 'presiden', 'Indonesia', 'se...","['Menteri', 'Sekretaris', 'Negara', 'Yusril', ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
8,"['Apakah', 'nama', 'kapsul', 'karya', 'Rusia',...","['Shenzhou', 'yang', 'dikembangkan', 'dari', '...","['O', 'O', 'O', 'O', 'O', 'B', 'O', 'O', 'O', ..."
9,"['Apakah', 'istilah', 'untuk', 'hasil', 'penca...","['Biodiesel', 'berpotensi', 'menggantikan', 's...","['O', 'O', 'O', 'O', 'O', 'B', 'O', 'O', 'O', ..."


In [46]:
def get_qa(filename, prefix='id-'):
    data_df = pd.read_csv(filename, names=['question','passage','seq_label'], header=0)
    data_qa = []
    for i, row in data_df.iterrows():
        #if i>5:
        #    break
        question_list = [q.strip("\'") for q in row['question'].strip('[]').split(', ')]
        question = ' '.join(question_list)
        passage_list = [p.strip("\'") for p in row['passage'].strip('[]').split(', ')]
        context = ' '.join(passage_list)
        seq_label_list = [s.strip("\'") for s in row['seq_label'].strip('[]').split(', ')]
        answer = []
        for j, label in enumerate(seq_label_list):
            if label != 'O':
                answer.append(passage_list[j])
            elif len(answer) != 0:
                break
        answer = ' '.join(answer)
        index = context.index(answer)
        #print(f'Q: {question}')
        #print(f'C: {context}')
        #print(f'A: {answer}, I:{index}\n')
        data = {'qas': [{'question': question, 'answers': [{'text': answer, 'answer_start': index}], 'id':f'{prefix}{i}' }],
               'context': context}
        data_qa.append(data)
    return data_qa

In [47]:
data_train_qa = get_qa(file_train, 'train-')
data_valid_qa = get_qa(file_valid, 'valid-')
data_test_qa = get_qa(file_test, 'test-')

In [48]:
data_train_qa[:2]

[{'qas': [{'question': 'Kelompok apakah yang menyatakan bertanggung jawab atas ledakan di Srinagar ?',
    'answers': [{'text': 'Hezb-ul Mujahedeen', 'answer_start': 60}],
    'id': 'train-0'}],
  'context': 'Lewat telepon ke kantor berita lokal Current News Service , Hezb-ul Mujahedeen , kelompok militan Kashmir yang terbesar , menyatakan bertanggung jawab atas ledakan di Srinagar .'},
 {'qas': [{'question': 'Di pulau manakah , PT Mutiara Indopearl berlokasi sebelum dekade 2000 - an',
    'answers': [{'text': 'Pulau Makassar', 'answer_start': 108}],
    'id': 'train-1'}],
  'context': 'Sejak tahun 1980 - an di Kota Bau-Bau terdapat dua perusahaan budidaya mutiara , yaitu PT Mutiara Indopearl Pulau Makassar dan PT Selat Buton . PT Mutiara Indopearl berlokasi di Pulau Makassar dan PT Selat Buton di Palabusa .'}]

In [49]:
data_valid_qa[:2]

[{'qas': [{'question': 'Apa nama latin dari harimau Sumatera ?',
    'answers': [{'text': 'Panthera tigris sumatrensis', 'answer_start': 85}],
    'id': 'valid-0'}],
  'context': 'Selain orangutan , satwa yang berada TNGL itu di antaranya adalah harimau sumatera ( Panthera tigris sumatrensis ) . Tim Kompas bulan lalu pernah memergoki harimau ini di tepi jalan raya Tapaktuan-Singkil , tepatnya di Desa Sultan Daulat , yang masuk koridor ekosistem Leuser .'},
 {'qas': [{'question': 'Berapa penumpang yang turut dalam penerbangan GA - 181 jurusan Medan-Jakarta yang mendarat darurat di Bandar Udara Sultan Syarif Kasim II Pekanbaru , Riau pada tanggal 6 September 2005 ?',
    'answers': [{'text': '84 penumpang', 'answer_start': 55}],
    'id': 'valid-1'}],
  'context': 'Pesawat dengan nomor penerbangan GA - 181 yang membawa 84 penumpang , termasuk Menteri Sosial Bachtiar Chamsyah dan sejumlah anggota Komisi V DPR , ini terpaksa mendarat darurat karena terjadi kerusakan pada mesin di sayap kan

In [50]:
data_test_qa[:2]

[{'qas': [{'question': 'Siapakah pelatih ganda putra bulu tangkis yang tidak meragukan tekad Candra / Sigit untuk bekerja lebih keras pada perebutan piala Thomas',
    'answers': [{'text': '', 'answer_start': 0}],
    'id': 'test-0'}],
  'context': 'Pelatih ganda putra , Christian Hadinata , tak meragukan tekad Candra / Sigit . Bila dia sudah ngomong begitu , saya tidak ragu lagi , ujarnya .'},
 {'qas': [{'question': 'Siapa nama Perdana Menteri Inggris ?',
    'answers': [{'text': '', 'answer_start': 0}],
    'id': 'test-1'}],
  'context': 'Curtis mengatakan hal itu setelah membandingkan janji-janji besar G - 8 dengan kenyataan di lapangan . Pada Juli lalu , Perdana Menteri Inggris Tony Blair mengatakan , G - 8 setuju menaikkan bantuan sebesar 48 miliar dollar AS per tahun pada 2010 .'}]

In [51]:
len(data_train_qa), len(data_valid_qa), len(data_test_qa)

(2495, 311, 311)

In [12]:
# Configure the model
model_args = QuestionAnsweringArgs()
model_args.num_train_epochs = 5
model_args.train_batch_size = 64
model_args.evaluate_during_training = True
model_args.output_dir = '/output/indonlu/facqa/bert-base-indonesian-522M'
model_args.best_model_dir = f'{model_args.output_dir}/best_model'
model_args.overwrite_output_dir = True
model_args.fp16 = False
model_args.do_lower_case = True

In [13]:
model = QuestionAnsweringModel(
    "bert", "cahya/bert-base-indonesian-522M", args=model_args
    #"roberta", "cahya/roberta-base-indonesian-522M", args=model_args,
    #"xlmroberta", "xlm-roberta-base", args=model_args,
    #"bert", "/output/bert-id-100/bert-base-indonesian-522M-100e", args=model_args
)

Some weights of the model checkpoint at cahya/bert-base-indonesian-522M were not used when initializing BertForQuestionAnswering: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at cahya/bert-base-indonesian-522M a

In [14]:
# Train the model
model.train_model(data_train_qa, eval_data=data_valid_qa)


INFO:simpletransformers.question_answering.question_answering_model: Converting to features started.
convert squad examples to features:   0%|          | 0/2495 [00:00<?, ?it/s]/root/Work/transformers/src/transformers/tokenization_utils_base.py:1321: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,
/root/Work/transformers/src/transformers/tokenization_utils_base.py:1321: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,
/root/Work/transformers/src/transformers/tokenization_utils_base.py:1321: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,
/root/Work/transformers/src/transformers/tokenization_utils_base.py:1321: FutureWarning: The `max_len` attribute has been deprecated and will be removed i

/sysadmin/wirawan/miniconda3/envs/transformers-cuda9/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)
INFO:simpletransformers.question_answering.question_answering_model: Converting to features started.

convert squad examples to features:   0%|          | 0/311 [00:00<?, ?it/s]/root/Work/transformers/src/transformers/tokenization_utils_base.py:1321: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,

convert squad examples to features: 100%|██████████| 311/311 [00:01<00:00, 234.54it/s]

add example index and unique id: 100%|██████████| 311/311 [00:00<00:00, 302230.90it/s]


/sysadmin/wirawan/miniconda3/envs/transformers-cuda9/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:231: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


INFO:simpletransformers.question_answering.question_answering_model: Converting to features started.

convert squad examples to features:   0%|          | 0/311 [00:00<?, ?it/s]/root/Work/transformers/src/transformers/tokenization_utils_base.py:1321: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,

convert squad examples to features: 100%|██████████| 311/311 [00:01<00:00, 232.33it/s]

add example index and unique id: 100%|██████████| 311/311 [00:00<00:00, 368899.48it/s]


INFO:simpletransformers.question_answering.question_answering_model: Converting to features started.

convert squad examples to features:   0%|          | 0/311 [00:00<?, ?it/s]/root/Work/transformers/src/transformers/tokenization_utils_base.py:1321: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,

convert squad examples to features: 100%|██████████| 311/311 [00:01<00:00, 221.22it/s]

add example index and unique id: 100%|██████████| 311/311 [00:00<00:00, 408655.56it/s]


INFO:simpletransformers.question_answering.question_answering_model: Converting to features started.

convert squad examples to features:   0%|          | 0/311 [00:00<?, ?it/s]/root/Work/transformers/src/transformers/tokenization_utils_base.py:1321: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,

convert squad examples to features: 100%|██████████| 311/311 [00:01<00:00, 225.43it/s]

add example index and unique id: 100%|██████████| 311/311 [00:00<00:00, 508747.48it/s]


INFO:simpletransformers.question_answering.question_answering_model: Converting to features started.

convert squad examples to features:   0%|          | 0/311 [00:00<?, ?it/s]/root/Work/transformers/src/transformers/tokenization_utils_base.py:1321: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,

convert squad examples to features: 100%|██████████| 311/311 [00:01<00:00, 229.61it/s]

add example index and unique id: 100%|██████████| 311/311 [00:00<00:00, 388801.35it/s]


INFO:simpletransformers.question_answering.question_answering_model: Training of bert model complete. Saved to /output/indonlu/facqa/bert-base-indonesian-522M.


In [15]:
# Evaluate the model
result, texts = model.eval_model(data_valid_qa)

INFO:simpletransformers.question_answering.question_answering_model: Converting to features started.
convert squad examples to features:   0%|          | 0/311 [00:00<?, ?it/s]/root/Work/transformers/src/transformers/tokenization_utils_base.py:1321: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,
add example index and unique id: 100%|██████████| 311/311 [00:00<00:00, 383655.45it/s]


In [24]:
result

{'correct': 0, 'similar': 311, 'incorrect': 0, 'eval_loss': -7.691952424171643}

In [17]:
# Evaluate the model
result, texts = model.eval_model(data_test_qa)

INFO:simpletransformers.question_answering.question_answering_model: Converting to features started.
convert squad examples to features:   0%|          | 0/311 [00:00<?, ?it/s]/root/Work/transformers/src/transformers/tokenization_utils_base.py:1321: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,
add example index and unique id: 100%|██████████| 311/311 [00:00<00:00, 332338.48it/s]


In [18]:
result

{'correct': 0, 'similar': 311, 'incorrect': 0, 'eval_loss': -7.691952424171643}

In [25]:
texts

{'correct_text': {},
 'similar_text': {'id-0': {'truth': '',
   'predicted': 'Christian Hadinata',
   'question': 'Siapakah pelatih ganda putra bulu tangkis yang tidak meragukan tekad Candra / Sigit untuk bekerja lebih keras pada perebutan piala Thomas'},
  'id-1': {'truth': '',
   'predicted': 'Tony Blair',
   'question': 'Siapa nama Perdana Menteri Inggris ?'},
  'id-2': {'truth': '',
   'predicted': '49,07 detik',
   'question': 'Berapakah catatan waktu terbaik Tonique Williams-Darling dalam lari 400 meter putri ?'},
  'id-3': {'truth': '',
   'predicted': 'Muntiacus muntjak',
   'question': 'Apakah nama latin dari kijang'},
  'id-4': {'truth': '',
   'predicted': 'Jusuf Kalla',
   'question': 'Siapa nama presiden Indonesia sekarang ?'},
  'id-5': {'truth': '',
   'predicted': 'Balai Kartini , Jakarta',
   'question': 'Dimanakah konser untuk mendiang Teguh Karya akan dilaksanakan ?'},
  'id-6': {'truth': '',
   'predicted': 'fumigasi',
   'question': 'Apa nama proses yang dilaksanak

In [29]:
data_test_qa[:5]

[{'qas': [{'question': 'Siapakah pelatih ganda putra bulu tangkis yang tidak meragukan tekad Candra / Sigit untuk bekerja lebih keras pada perebutan piala Thomas',
    'answers': [{'text': '', 'answer_start': 0}],
    'id': 'id-0'}],
  'context': 'Pelatih ganda putra , Christian Hadinata , tak meragukan tekad Candra / Sigit . Bila dia sudah ngomong begitu , saya tidak ragu lagi , ujarnya .'},
 {'qas': [{'question': 'Siapa nama Perdana Menteri Inggris ?',
    'answers': [{'text': '', 'answer_start': 0}],
    'id': 'id-1'}],
  'context': 'Curtis mengatakan hal itu setelah membandingkan janji-janji besar G - 8 dengan kenyataan di lapangan . Pada Juli lalu , Perdana Menteri Inggris Tony Blair mengatakan , G - 8 setuju menaikkan bantuan sebesar 48 miliar dollar AS per tahun pada 2010 .'},
 {'qas': [{'question': 'Berapakah catatan waktu terbaik Tonique Williams-Darling dalam lari 400 meter putri ?',
    'answers': [{'text': '', 'answer_start': 0}],
    'id': 'id-2'}],
  'context': 'Catatan w

In [26]:
def save_result(filename, data_test_qa, texts):
    prediction = {'index':[], 'label':[]}
    for i, qa in enumerate(data_test_qa):
        id = qa['qas'][0]['id']
        question = qa['qas'][0]['question']    
        context = qa['context']
        label = ['O']*len(context.split())
        try:
            answer = texts['similar_text'][id]['predicted']
            answer_list = answer.split()
            answer_start = context.index(answer)
            context_list = context[:answer_start].split()
            answer_index = len(context_list)
            for j, ans in enumerate(answer_list):
                if j == 0:
                    label[answer_index+j] = 'B'
                else:
                    label[answer_index+j] = 'I'
        except KeyError:
            pass
        prediction['index'].append(i)
        prediction['label'].append(str(label))
    prediction_df = pd.DataFrame.from_dict(prediction).set_index('index')
    prediction_df.to_csv(filename)

In [28]:
texts['similar_text']['id-121']['predicted']

'PBB'

In [29]:
save_result(f'{model_args.output_dir}/result.csv', data_test_qa, texts)

## BERT-base-indonesian-1.5G

In [30]:
# Configure the model
model_args = QuestionAnsweringArgs()
model_args.num_train_epochs = 5
model_args.train_batch_size = 64
model_args.evaluate_during_training = True
model_args.output_dir = '/output/indonlu/facqa/bert-base-indonesian-1.5G'
model_args.best_model_dir = f'{model_args.output_dir}/best_model'
model_args.overwrite_output_dir = True
model_args.fp16 = False
model_args.do_lower_case = True

In [31]:
model = QuestionAnsweringModel(
    "bert", "cahya/bert-base-indonesian-1.5G", args=model_args
    #"roberta", "cahya/roberta-base-indonesian-522M", args=model_args,
    #"xlmroberta", "xlm-roberta-base", args=model_args,
    #"bert", "/output/bert-id-100/bert-base-indonesian-522M-100e", args=model_args
)

Some weights of the model checkpoint at cahya/bert-base-indonesian-1.5G were not used when initializing BertForQuestionAnswering: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at cahya/bert-base-indonesian-1.5G a

In [32]:
# Train the model
model.train_model(data_train_qa, eval_data=data_valid_qa)

INFO:simpletransformers.question_answering.question_answering_model: Converting to features started.
convert squad examples to features:   0%|          | 0/2495 [00:00<?, ?it/s]/root/Work/transformers/src/transformers/tokenization_utils_base.py:1321: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,
/root/Work/transformers/src/transformers/tokenization_utils_base.py:1321: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,
/root/Work/transformers/src/transformers/tokenization_utils_base.py:1321: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,
/root/Work/transformers/src/transformers/tokenization_utils_base.py:1321: FutureWarning: The `max_len` attribute has been deprecated and will be removed i

INFO:simpletransformers.question_answering.question_answering_model: Converting to features started.

convert squad examples to features:   0%|          | 0/311 [00:00<?, ?it/s]/root/Work/transformers/src/transformers/tokenization_utils_base.py:1321: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,

convert squad examples to features: 100%|██████████| 311/311 [00:01<00:00, 229.77it/s]

add example index and unique id: 100%|██████████| 311/311 [00:00<00:00, 549927.72it/s]


INFO:simpletransformers.question_answering.question_answering_model: Converting to features started.

convert squad examples to features:   0%|          | 0/311 [00:00<?, ?it/s]/root/Work/transformers/src/transformers/tokenization_utils_base.py:1321: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,

convert squad examples to features: 100%|██████████| 311/311 [00:01<00:00, 171.76it/s]

add example index and unique id: 100%|██████████| 311/311 [00:00<00:00, 544874.08it/s]


INFO:simpletransformers.question_answering.question_answering_model: Converting to features started.

convert squad examples to features:   0%|          | 0/311 [00:00<?, ?it/s]/root/Work/transformers/src/transformers/tokenization_utils_base.py:1321: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,

convert squad examples to features: 100%|██████████| 311/311 [00:01<00:00, 224.51it/s]

add example index and unique id: 100%|██████████| 311/311 [00:00<00:00, 541256.66it/s]


INFO:simpletransformers.question_answering.question_answering_model: Converting to features started.

convert squad examples to features:   0%|          | 0/311 [00:00<?, ?it/s]/root/Work/transformers/src/transformers/tokenization_utils_base.py:1321: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,

convert squad examples to features: 100%|██████████| 311/311 [00:01<00:00, 235.57it/s]

add example index and unique id: 100%|██████████| 311/311 [00:00<00:00, 430931.13it/s]


INFO:simpletransformers.question_answering.question_answering_model: Converting to features started.

convert squad examples to features:   0%|          | 0/311 [00:00<?, ?it/s]/root/Work/transformers/src/transformers/tokenization_utils_base.py:1321: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,

convert squad examples to features: 100%|██████████| 311/311 [00:01<00:00, 225.39it/s]

add example index and unique id: 100%|██████████| 311/311 [00:00<00:00, 513959.24it/s]


INFO:simpletransformers.question_answering.question_answering_model: Training of bert model complete. Saved to /output/indonlu/facqa/bert-base-indonesian-1.5G.


In [33]:
# Evaluate the model
result, texts = model.eval_model(data_valid_qa)

INFO:simpletransformers.question_answering.question_answering_model: Converting to features started.
convert squad examples to features:   0%|          | 0/311 [00:00<?, ?it/s]/root/Work/transformers/src/transformers/tokenization_utils_base.py:1321: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,
add example index and unique id: 100%|██████████| 311/311 [00:00<00:00, 356986.47it/s]


In [34]:
result

{'correct': 216,
 'similar': 57,
 'incorrect': 38,
 'eval_loss': -7.261089691748986}

In [35]:
# Evaluate the model
result, texts = model.eval_model(data_test_qa)

INFO:simpletransformers.question_answering.question_answering_model: Converting to features started.
convert squad examples to features:   0%|          | 0/311 [00:00<?, ?it/s]/root/Work/transformers/src/transformers/tokenization_utils_base.py:1321: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,
add example index and unique id: 100%|██████████| 311/311 [00:00<00:00, 37468.51it/s]


In [36]:
save_result(f'{model_args.output_dir}/result.csv', data_test_qa, texts)

### Dataset training+valid

In [57]:
data_train_valid_qa = data_train_qa+data_valid_qa

In [58]:
len(data_train_qa), len(data_valid_qa), len(data_train_valid_qa)

(2495, 311, 2806)

In [61]:
# Configure the model
model_args = QuestionAnsweringArgs()
model_args.num_train_epochs = 5
model_args.train_batch_size = 64
model_args.evaluate_during_training = True
model_args.output_dir = '/output/indonlu/facqa/bert-base-indonesian-1.5G'
model_args.best_model_dir = f'{model_args.output_dir}/best_model'
model_args.overwrite_output_dir = True
model_args.fp16 = False
model_args.do_lower_case = True

In [62]:
model = QuestionAnsweringModel(
    "bert", "cahya/bert-base-indonesian-1.5G", args=model_args
    #"roberta", "cahya/roberta-base-indonesian-522M", args=model_args,
    #"xlmroberta", "xlm-roberta-base", args=model_args,
    #"bert", "/output/bert-id-100/bert-base-indonesian-522M-100e", args=model_args
)

Some weights of the model checkpoint at cahya/bert-base-indonesian-1.5G were not used when initializing BertForQuestionAnswering: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at cahya/bert-base-indonesian-1.5G a

In [63]:
# Train the model
model.train_model(data_train_valid_qa, eval_data=data_valid_qa)

INFO:simpletransformers.question_answering.question_answering_model: Converting to features started.
convert squad examples to features:   0%|          | 0/2806 [00:00<?, ?it/s]/root/Work/transformers/src/transformers/tokenization_utils_base.py:1321: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,
/root/Work/transformers/src/transformers/tokenization_utils_base.py:1321: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,
/root/Work/transformers/src/transformers/tokenization_utils_base.py:1321: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,
/root/Work/transformers/src/transformers/tokenization_utils_base.py:1321: FutureWarning: The `max_len` attribute has been deprecated and will be removed i

INFO:simpletransformers.question_answering.question_answering_model: Converting to features started.

convert squad examples to features:   0%|          | 0/311 [00:00<?, ?it/s]/root/Work/transformers/src/transformers/tokenization_utils_base.py:1321: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,

convert squad examples to features: 100%|██████████| 311/311 [00:01<00:00, 230.19it/s]

add example index and unique id: 100%|██████████| 311/311 [00:00<00:00, 532202.59it/s]


INFO:simpletransformers.question_answering.question_answering_model: Converting to features started.

convert squad examples to features:   0%|          | 0/311 [00:00<?, ?it/s]/root/Work/transformers/src/transformers/tokenization_utils_base.py:1321: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,

convert squad examples to features: 100%|██████████| 311/311 [00:02<00:00, 154.30it/s]

add example index and unique id: 100%|██████████| 311/311 [00:00<00:00, 523656.58it/s]


INFO:simpletransformers.question_answering.question_answering_model: Converting to features started.

convert squad examples to features:   0%|          | 0/311 [00:00<?, ?it/s]/root/Work/transformers/src/transformers/tokenization_utils_base.py:1321: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,

convert squad examples to features: 100%|██████████| 311/311 [00:01<00:00, 236.09it/s]

add example index and unique id: 100%|██████████| 311/311 [00:00<00:00, 428806.23it/s]


INFO:simpletransformers.question_answering.question_answering_model: Converting to features started.

convert squad examples to features:   0%|          | 0/311 [00:00<?, ?it/s]/root/Work/transformers/src/transformers/tokenization_utils_base.py:1321: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,

convert squad examples to features: 100%|██████████| 311/311 [00:01<00:00, 231.43it/s]

add example index and unique id: 100%|██████████| 311/311 [00:00<00:00, 500548.17it/s]


INFO:simpletransformers.question_answering.question_answering_model: Converting to features started.

convert squad examples to features:   0%|          | 0/311 [00:00<?, ?it/s]/root/Work/transformers/src/transformers/tokenization_utils_base.py:1321: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,

convert squad examples to features: 100%|██████████| 311/311 [00:01<00:00, 222.22it/s]

add example index and unique id: 100%|██████████| 311/311 [00:00<00:00, 502282.84it/s]


INFO:simpletransformers.question_answering.question_answering_model: Training of bert model complete. Saved to /output/indonlu/facqa/bert-base-indonesian-1.5G.


In [64]:
# Evaluate the model
result, texts = model.eval_model(data_valid_qa)

INFO:simpletransformers.question_answering.question_answering_model: Converting to features started.
convert squad examples to features:   0%|          | 0/311 [00:00<?, ?it/s]/root/Work/transformers/src/transformers/tokenization_utils_base.py:1321: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,
add example index and unique id: 100%|██████████| 311/311 [00:00<00:00, 353025.32it/s]


In [66]:
result

{'correct': 306, 'similar': 5, 'incorrect': 0, 'eval_loss': -7.844043230399107}

In [67]:
# Evaluate the model
result, texts = model.eval_model(data_test_qa)

INFO:simpletransformers.question_answering.question_answering_model: Converting to features started.
convert squad examples to features:   0%|          | 0/311 [00:00<?, ?it/s]/root/Work/transformers/src/transformers/tokenization_utils_base.py:1321: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,
add example index and unique id: 100%|██████████| 311/311 [00:00<00:00, 405227.88it/s]


In [68]:
save_result(f'{model_args.output_dir}/result_t_v.csv', data_test_qa, texts)

### tydiqa + facqa

In [74]:
file_train_tq = "/dataset/tydiqa/tydiqa-goldp-v1.1-train.json"
file_valid_tq = "/dataset/tydiqa/tydiqa-goldp-v1.1-dev.json"

In [76]:
with open(file_valid_tq) as f:
  data_valid_tq = json.load(f)

with open(file_train_tq) as f:
  data_train_tq = json.load(f)

In [77]:
data_valid_tq_id = []
for i in range(len(data_valid_tq["data"])):
    if 'indonesia' in data_valid_tq["data"][i]['paragraphs'][0]['qas'][0]['id']:
        for paragraph in data_valid_tq["data"][i]['paragraphs']:
            data_valid_tq_id.append(paragraph)

In [78]:
data_train_tq_id = []
for i in range(len(data_train_tq["data"])):
    if 'indonesia' in data_train_tq["data"][i]['paragraphs'][0]['qas'][0]['id']:
        for paragraph in data_train_tq["data"][i]['paragraphs']:
            data_train_tq_id.append(paragraph)

In [79]:
data_valid_tq_id[:2]

[{'qas': [{'question': 'Siapakah yang menemuka benua Amerika ?',
    'answers': [{'text': 'orang-orang Viking dari Eropa Utara',
      'answer_start': 193}],
    'id': 'indonesian-1906590851264221380-1'}],
  'context': "Kolumbus bukanlah orang pertama yang tiba di Amerika, yang ia dapati sudah diduduki. Ia juga bukan orang Eropa pertama yang sampai ke benua itu karena sekarang telah diakui secara meluas bahwa orang-orang Viking dari Eropa Utara telah berkunjung ke Amerika Utara pada abad ke 11 dan mendirikan koloni L'Anse aux Meadows untuk jangka waktu singkat. Terdapat perkiraan bahwa pelayar yang tidak dikenali pernah melawat ke Amerika sebelum Kolumbus dan membekalkannya dengan sumber untuk kejayaannya. Terdapat juga banyak teori mengenai ekspedisi ke Amerika oleh berbagai orang sepanjang masa itu."},
 {'qas': [{'question': 'Dimanakah letak Donggala ?',
    'answers': [{'text': 'provinsi Sulawesi Tengah, Indonesia',
      'answer_start': 75},
     {'text': 'Sulawesi Tengah, Indonesi

In [93]:
#data_all_qa = data_train_qa+data_valid_qa+data_train_tq_id+data_valid_tq_id
data_all_qa = data_train_qa+data_train_tq_id+data_valid_tq_id

In [94]:
len(data_train_qa), len(data_valid_qa), len(data_train_tq_id), len(data_valid_tq_id), len(data_all_qa)

(2495, 311, 5702, 565, 8762)

In [85]:
data_all_qa[2800:2810]

[{'qas': [{'question': 'Kapankah pelaksanaan Tokyo Performing Arts Market yang sedianya akan dihadiri oleh Dwiki Dharmawan ?',
    'answers': [{'text': '7-19 September', 'answer_start': 276}],
    'id': 'valid-305'}],
  'context': 'Pemusik Dwiki Dharmawan ( 39 ) , Rabu ( 7/9 ) , kehilangan tas yang berisi dokumen penting , termasuk paspor , tiket pesawat Jakarta-Tokyo , KTP , sampai telepon seluler . Padahal , malam harinya ia harus berangkat ke Jepang untuk menghadiri Tokyo Performing Arts Market pada 7-19 September .'},
 {'qas': [{'question': 'Siapa Wakil Presiden Indonesia ?',
    'answers': [{'text': 'Jusuf Kalla', 'answer_start': 38}],
    'id': 'valid-306'}],
  'context': 'Ada imbauan dari Bapak Wakil Presiden Jusuf Kalla bahwa saat ini memang dibutuhkan keputusan yang cepat dan tepat dalam hal ekonomi . Kami sudah sering sekali berdialog dengan kalangan industri dan kami tahu kalau industri keberatan jika tarif listriknya dinaikkan . Namun , melihat jenis risiko yang dipilih pem

In [95]:
# Configure the model
model_args = QuestionAnsweringArgs()
model_args.num_train_epochs = 5
model_args.train_batch_size = 64
model_args.evaluate_during_training = True
model_args.output_dir = '/output/indonlu/facqa/bert-base-indonesian-1.5G'
model_args.best_model_dir = f'{model_args.output_dir}/best_model'
model_args.overwrite_output_dir = True
model_args.fp16 = False
model_args.do_lower_case = True

In [96]:
model = QuestionAnsweringModel(
    #"bert", "cahya/bert-base-indonesian-1.5G", args=model_args
    #"roberta", "cahya/roberta-base-indonesian-522M", args=model_args,
    "xlmroberta", "xlm-roberta-base", args=model_args,
    #"bert", "/output/bert-id-100/bert-base-indonesian-522M-100e", args=model_args
)

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForQuestionAnswering: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing XLMRobertaForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing XLMRobertaForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForQuestionAnswering were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream tas

In [97]:
# Train the model
model.train_model(data_all_qa, eval_data=data_valid_qa)

INFO:simpletransformers.question_answering.question_answering_model: Converting to features started.
convert squad examples to features:   0%|          | 0/8762 [00:00<?, ?it/s]/root/Work/transformers/src/transformers/tokenization_utils_base.py:1321: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,
/root/Work/transformers/src/transformers/tokenization_utils_base.py:1321: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,
/root/Work/transformers/src/transformers/tokenization_utils_base.py:1321: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,
/root/Work/transformers/src/transformers/tokenization_utils_base.py:1321: FutureWarning: The `max_len` attribute has been deprecated and will be removed i

INFO:simpletransformers.question_answering.question_answering_model: Converting to features started.

convert squad examples to features:   0%|          | 0/311 [00:00<?, ?it/s]/root/Work/transformers/src/transformers/tokenization_utils_base.py:1321: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,

convert squad examples to features: 100%|██████████| 311/311 [00:01<00:00, 306.31it/s]

add example index and unique id: 100%|██████████| 311/311 [00:00<00:00, 444590.51it/s]


INFO:simpletransformers.question_answering.question_answering_model: Converting to features started.

convert squad examples to features:   0%|          | 0/311 [00:00<?, ?it/s]/root/Work/transformers/src/transformers/tokenization_utils_base.py:1321: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,

convert squad examples to features: 100%|██████████| 311/311 [00:01<00:00, 300.10it/s]

add example index and unique id: 100%|██████████| 311/311 [00:00<00:00, 524920.94it/s]


INFO:simpletransformers.question_answering.question_answering_model: Converting to features started.

convert squad examples to features:   0%|          | 0/311 [00:00<?, ?it/s]/root/Work/transformers/src/transformers/tokenization_utils_base.py:1321: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,

convert squad examples to features: 100%|██████████| 311/311 [00:01<00:00, 301.40it/s]

add example index and unique id: 100%|██████████| 311/311 [00:00<00:00, 406237.48it/s]


INFO:simpletransformers.question_answering.question_answering_model: Converting to features started.

convert squad examples to features:   0%|          | 0/311 [00:00<?, ?it/s]/root/Work/transformers/src/transformers/tokenization_utils_base.py:1321: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,

convert squad examples to features: 100%|██████████| 311/311 [00:01<00:00, 280.85it/s]

add example index and unique id: 100%|██████████| 311/311 [00:00<00:00, 509542.40it/s]


INFO:simpletransformers.question_answering.question_answering_model: Converting to features started.

convert squad examples to features:   0%|          | 0/311 [00:00<?, ?it/s]/root/Work/transformers/src/transformers/tokenization_utils_base.py:1321: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,

convert squad examples to features: 100%|██████████| 311/311 [00:01<00:00, 307.53it/s]

add example index and unique id: 100%|██████████| 311/311 [00:00<00:00, 442329.11it/s]


INFO:simpletransformers.question_answering.question_answering_model: Training of xlmroberta model complete. Saved to /output/indonlu/facqa/bert-base-indonesian-1.5G.


In [98]:
# Evaluate the model
result, texts = model.eval_model(data_valid_qa)

INFO:simpletransformers.question_answering.question_answering_model: Converting to features started.
convert squad examples to features:   0%|          | 0/311 [00:00<?, ?it/s]/root/Work/transformers/src/transformers/tokenization_utils_base.py:1321: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,
add example index and unique id: 100%|██████████| 311/311 [00:00<00:00, 388569.72it/s]


In [99]:
result

{'correct': 209,
 'similar': 79,
 'incorrect': 23,
 'eval_loss': -7.259224671583909}

In [100]:
# Evaluate the model
result, texts = model.eval_model(data_test_qa)

INFO:simpletransformers.question_answering.question_answering_model: Converting to features started.
convert squad examples to features:   0%|          | 0/311 [00:00<?, ?it/s]/root/Work/transformers/src/transformers/tokenization_utils_base.py:1321: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,
add example index and unique id: 100%|██████████| 311/311 [00:00<00:00, 421872.10it/s]


In [101]:
save_result(f'{model_args.output_dir}/result_fq_t_tq_tv.csv', data_test_qa, texts)

In [69]:
# Make predictions with the model
to_predict = [
    {
        "context": "Presiden Suharto adalah Presiden kedua Indonesia. Presiden Suharto wafat dini hari tanggal 1 Agustus 1999 (tertulis di batu nisannya; 2 Agustus 1999 versi van der Veur, 2006) dan dimakamkan di Taman Kopo, Cirebon. ",
        "qas": [
            {
                "question": "Kapan Presiden Suharto meninggal dunia?",
                "id": "0",
            },
            {
                "question": "dimana Presiden Suharto dikuburkan?",
                "id": "1",
            },
            {
                "question": "Siapa Presiden Suharto?",
                "id": "2",
            }
        ],
    }
]
to_predict2 = [
    {
        "context": "Mikrohidro atau yang dimaksud dengan Pembangkit Listrik Tenaga Mikrohidro (PLTMH), adalah suatu pembangkit listrik skala kecil yang menggunakan tenaga air sebagai tenaga penggeraknya seperti, saluran irigasi, sungai atau air terjun alam dengan cara memanfaatkan tinggi terjunan dan jumlah debit air. Mikrohidro merupakan sebuah istilah yang terdiri dari kata mikro yang berarti kecil dan hidro yang berarti air. Secara teknis, mikrohidro memiliki tiga komponen utama yaitu air, turbin dan generator. Mikrohidro mendapatkan energi dari aliran air yang memiliki perbedaan ketinggian tertentu.",
        "qas": [
            {
                "question": "Apa Mikrohidro itu?",
                "id": "0",
            },
            {
                "question": "apa komponen mikrohidro?",
                "id": "1",
            },
            {
                "question": "dimana Mikrohidro mendapatkan energi?",
                "id": "2",
            }
        ],
    }
]



In [70]:
answers, probabilities = model.predict(to_predict2)

INFO:simpletransformers.question_answering.question_answering_model: Converting to features started.
convert squad examples to features:   0%|          | 0/3 [00:00<?, ?it/s]/root/Work/transformers/src/transformers/tokenization_utils_base.py:1321: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,
add example index and unique id: 100%|██████████| 3/3 [00:00<00:00, 8659.95it/s]


In [71]:
print(answers[:][0]['answer'][:3], probabilities)

['Pembangkit Listrik Tenaga Mikrohidro (PLTMH),', '(PLTMH),', 'hidro'] [{'id': '0', 'probability': [0.2851562179649796, 0.13139844865615938, 0.12066437720254321, 0.09407750028542083, 0.07168729305250356, 0.05122965426303453, 0.043350405189031646, 0.03303311834713446, 0.03260847048438781, 0.024031920145636296, 0.02276322651833435, 0.017747636037444885, 0.015025807486424618, 0.01189447720086393, 0.00927367884542169, 0.008178015051808561, 0.007928506663803379, 0.007475387107253625, 0.00643294783120816, 0.006041542121680722]}, {'id': '1', 'probability': [0.1617394043446816, 0.14344205655627984, 0.1288480887330671, 0.11938283732596382, 0.05087100957056073, 0.04339116776701392, 0.038482386936549276, 0.03202782119732323, 0.03191271497483673, 0.031751331677356935, 0.02830247511317419, 0.028159348877621834, 0.026375957127316482, 0.02542294716275831, 0.023555363492941284, 0.02343624350466002, 0.01737081624989311, 0.01710313159388067, 0.01506448410123257, 0.013360260532647598]}, {'id': '2', 'prob

In [38]:
answers

[{'id': '0',
  'answer': ['pembangkit listrik skala kecil yang menggunakan tenaga air sebagai tenaga penggeraknya seperti, saluran irigasi, sungai atau air terjun alam dengan cara memanfaatkan tinggi terjunan dan jumlah debit air',
   'suatu pembangkit listrik skala kecil yang menggunakan tenaga air sebagai tenaga penggeraknya seperti, saluran irigasi, sungai atau air terjun alam dengan cara memanfaatkan tinggi terjunan dan jumlah debit air',
   'pembangkit listrik skala kecil yang menggunakan tenaga air sebagai tenaga penggeraknya seperti, saluran irigasi',
   'pembangkit listrik skala kecil yang menggunakan tenaga air sebagai tenaga penggeraknya seperti, saluran irigasi, sungai atau air terjun alam dengan cara memanfaatkan tinggi terjunan dan jumlah debit air. Mikrohidro merupakan sebuah istilah yang terdiri dari kata mikro yang berarti kecil dan hidro yang berarti air.',
   'skala kecil yang menggunakan tenaga air sebagai tenaga penggeraknya seperti, saluran irigasi, sungai atau air

In [52]:
for i, answer in enumerate(answers):
    print(i, to_predict2[0]['qas'][i]['question'], ":", answer['answer'][:3], probabilities[i]['probability'][0])

0 Apa Mikrohidro itu? : ['pembangkit listrik skala kecil yang menggunakan tenaga air sebagai tenaga penggeraknya seperti, saluran irigasi, sungai atau air terjun alam dengan cara memanfaatkan tinggi terjunan dan jumlah debit air', 'suatu pembangkit listrik skala kecil yang menggunakan tenaga air sebagai tenaga penggeraknya seperti, saluran irigasi, sungai atau air terjun alam dengan cara memanfaatkan tinggi terjunan dan jumlah debit air', 'pembangkit listrik skala kecil yang menggunakan tenaga air sebagai tenaga penggeraknya seperti, saluran irigasi'] 0.9632559724729386
1 apa komponen mikrohidro? : ['tiga komponen utama yaitu air, turbin dan generator', 'utama yaitu air, turbin dan generator', 'pembangkit listrik skala kecil yang menggunakan tenaga air sebagai tenaga penggeraknya seperti, saluran irigasi, sungai atau air terjun alam dengan cara memanfaatkan tinggi terjunan dan jumlah debit air. Mikrohidro merupakan sebuah istilah yang terdiri dari kata mikro yang berarti kecil dan hidr